# Automate the boring stuff!
This work is licensed under CC BY-NC-SA 4.0 https://creativecommons.org/licenses/by-nc-sa/4.0/

Notebook ontwikkeld voor de belevingsdagen op Thomas More Mechelen.<br>
Interessant boek: https://automatetheboringstuff.com/

### Te installeren packages (zie PyPi https://pypi.org/)

In [ ]:
! pip install requests
! pip install bs4
! pip install smtplib
! pip install email

### In te vullen waarden

In [ ]:
URL = "https://nl.wikipedia.org/wiki/Albert_Einstein" # welke wikipedia-pagina wil je gebruiken
FOLDER = "./images/" #folder voor afbeeldingen
APIKEY = "" #API Key van, freemium https://rapidapi.com/rphrp1985/api/chatgpt-42"
GMAIL = "" #gmail-account
GMAIL_APP_PASSWORD = "" #gmail app password

## Webscrape informatie
We halen alle informatie van wikipedia binnen

In [ ]:
import requests
from bs4 import BeautifulSoup

response = requests.get(URL)
soup = BeautifulSoup(response.content, "html.parser")
text = soup.find(id="mw-content-text").get_text().replace("\n"," ")
title = soup.title.string

print(title)
print(text)

We halen alle foto's van wikipedia binnen

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import os

response = requests.get(URL)
soup = BeautifulSoup(response.content, "html.parser")

# Loop over alle img-tags
for img_tag in soup.find_all("img"):
    img_src = img_tag.get("src")

    # Analyseer de bron-URL om de bestandsnaam te verkrijgen.
    filename = os.path.basename(urlparse(img_src).path)

    try:
        # Download de afbeelding
        with open(os.path.join(FOLDER, filename), "wb") as f:
            img_url = urljoin(URL, img_src)
            response = requests.get(img_url)
            f.write(response.content)
            print("Gedownload", filename)
    except:
        pass

## Werken met bestanden & mappen
Haal de grootste afbeelding eruit

In [ ]:
import os

afbeelding = None
max_size = 0

for file in os.listdir(FOLDER):
    file_path = os.path.join(FOLDER, file)

    size = os.path.getsize(file_path)
    if size > max_size:
        afbeelding, max_size = file, size

print(f"Grootste bestand '{afbeelding}' behouden.")



## Werken met een API
Vat de tekst van wikipedia samen in 50 woorden.

In [ ]:
import requests

url = "https://chatgpt-42.p.rapidapi.com/conversationgpt4"

payload = {
	"messages": [
		{
			"role": "user",
			"content": "Maak een samenvatting van de volgende tekst in het Nederlands in 50 woorden. \n" + text
		}
	],
	"system_prompt": "",
	"temperature": 0.9,
	"top_k": 5,
	"top_p": 0.9,
	"max_tokens": 256,
	"web_access": False
}
headers = {
	"content-type": "application/json",
	"X-RapidAPI-Key": APIKEY,
	"X-RapidAPI-Host": "chatgpt-42.p.rapidapi.com"
}

response = requests.post(url, json=payload, headers=headers)
samenvatting = response.json()["result"]
print(samenvatting)

## Stuur een e-mail
Stuur een e-mail met samenvatting en foto naar alle mensen die je wilt ;)

In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage

recipients = ["",""] # mogen zoveel namen in als je zelf wilt, scheiden door een ,

msg = MIMEMultipart()
msg["Subject"] = title
msg["To"] = ", ".join(recipients)
msg["From"] = f"{GMAIL}@gmail.com"

tekstdeel = MIMEText(samenvatting, 'plain', 'utf-8')
msg.attach(tekstdeel)

afbeeldingspad = os.path.join(FOLDER, afbeelding)
with open(afbeeldingspad, 'rb') as f:
    afbeeldingsinhoud = f.read()

mime_subtype = 'jpeg' if afbeelding.lower().endswith('.jpg') or afbeelding.lower().endswith('.jpeg') else 'png'
afbeelding = MIMEImage(afbeeldingsinhoud, _subtype=mime_subtype)
msg.attach(afbeelding)

smtp_server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
smtp_server.login(GMAIL, GMAIL_APP_PASSWORD)
smtp_server.sendmail(msg["From"], recipients, msg.as_string())
smtp_server.quit()


## Verwijder afbeeldingen in map
Dit is een stukje helper-code!

In [ ]:
import os

for file in os.listdir(FOLDER):
    file_path = os.path.join(FOLDER, file)
    os.remove(file_path)
    print(f"Verwijderd: {file}")
   
print("Alle bestanden zijn verwijderd.")